In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch09. 7장 LangChain과 vectorDatabase를 활용한 RAG구현(UpstageEmbedding) </font></b>

# RAG 절차

- 1. 문서를 읽는다
    - %pip install -U -q docx2txt
- 2. 문서를 쪼갠다
    - %pip install -qU langchain-text-splitters
- 3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf.클라우드 저장
    - %pip install -q langchain-chroma
- 4. 질문을 이용해 유사도 검색

- 5. 유사도 검색한 문서를 LLM에 질문화 함께 전달하여 답변 얻음(랭체인 사용가능)
    - %pip install -q langchain 
    - https://smith.langchain.com 에서 key 생성. .env에 LANGCHAIN_API_KEY로 추가 (시간이 남을 때 추가적으로 할만한것)

# 0. 패키지 설치

In [ ]:
# 문서 읽어오기
# %pip install -U -q docx2txt

In [2]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 벡터DB(로컬DB) 어제의 chromada가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서 읽기(X)

In [5]:
%%time

from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
document = loader.load()

CPU times: total: 3.94 s
Wall time: 4.29 s


In [7]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)

## 2.1 1500 토큰씩 쪼개서 읽어오기

In [8]:
%%time

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter

loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')

# gpt-4, gpt-4o, gpt-4 turbo, gpt-4o-mini, embedding 모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name='cl100k_base', # 토큰을 세는 방식 이름
    chunk_size=1500,             # chunk 당 토큰 수 기준
    chunk_overlap=200            # 겹치게 하는것
    # separators = ['\n', '\n\n'] 파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)

CPU times: total: 3.92 s
Wall time: 5.85 s


In [9]:
len(documents) # chunk 갯수

180

In [14]:
# chunk 글자수

# dir(documents[0])
# documents[0].page_content
print([len(document.page_content) for document in documents])

[1699, 1656, 1641, 1650, 1738, 1442, 1287, 1535, 1325, 1619, 1596, 1588, 1566, 1639, 1622, 1559, 1612, 1638, 1573, 1465, 1436, 1609, 1456, 1497, 1635, 1606, 1533, 1649, 1662, 1595, 1603, 1678, 1595, 1637, 1601, 1539, 1561, 1594, 1693, 1708, 1657, 1627, 1636, 1659, 1667, 1595, 1491, 1485, 1645, 1709, 1629, 1617, 1495, 1626, 1612, 1620, 1609, 1576, 1636, 1602, 1556, 1563, 1600, 1616, 1643, 1691, 1635, 1685, 1621, 1631, 1609, 1605, 1603, 1604, 1698, 1686, 1702, 1612, 1539, 1558, 1651, 2060, 1562, 1606, 1557, 1648, 1594, 1615, 1766, 1651, 1690, 1576, 1536, 1553, 1638, 1685, 1693, 1694, 1664, 1529, 1627, 1703, 1675, 1546, 1585, 1687, 1679, 1714, 1603, 1655, 1648, 1495, 1531, 1562, 1594, 1646, 1543, 1449, 1593, 1559, 1521, 1473, 1519, 1545, 1668, 1700, 1692, 1655, 1648, 1741, 1670, 1628, 1639, 1623, 1638, 1642, 1666, 1658, 1594, 1591, 1561, 1641, 1498, 1610, 1567, 1613, 1636, 1619, 1531, 1496, 1702, 1598, 1579, 1627, 1559, 1585, 1665, 1565, 1616, 1564, 1612, 1535, 1512, 1557, 1576, 1628, 165

In [17]:
# chunk 글자수 최대값과 최소값

print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents[:-1]])) # 마지막건 나머지다보니 빼고 확인

2060
1287


## 2.2 1500 글자씩 쪼개서 읽어오기

In [1]:
%%time
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,   # 문서를 쪼갤 때 1500 글자씩 chunking
                                               chunk_overlap=200, 
                                               # separators=['\n\n', '\n', ' ', ''] 기본값
                                               )
# 재귀적으로 다음 순서대로 시도:
    # 1. \n\n : 문단구분
    # 2. \n   : 줄바꿈
    # 3. ' '  : 공백
    # 4. ''   : 최후에는 글자단위로 chunking
    
documents = loader.load_and_split(text_splitter=text_splitter)

CPU times: total: 3.81 s
Wall time: 4.03 s


In [2]:
print('chunk 갯수 :', len(documents))

chunk 갯수 : 193


In [4]:
# chunk들의 글자수

print([len(document.page_content) for document in documents])

[1463, 1421, 1482, 1487, 1479, 1408, 1457, 1495, 1467, 1446, 1487, 1456, 1467, 1351, 1392, 1362, 1402, 1470, 1410, 1489, 1455, 1496, 1441, 1319, 1458, 1476, 1452, 1382, 1384, 1467, 1227, 1494, 1494, 1470, 1454, 1495, 1412, 1477, 1477, 1362, 1449, 1386, 1055, 1467, 1361, 1493, 1467, 1434, 1351, 1471, 1495, 1479, 1457, 1442, 1370, 873, 1419, 1357, 1353, 1316, 1349, 1452, 1439, 1363, 1433, 1412, 1306, 1200, 1411, 1452, 1421, 1318, 1416, 1333, 1308, 1385, 1479, 1495, 1399, 1375, 1360, 1353, 1382, 1446, 1356, 1409, 1483, 1486, 1157, 1233, 1443, 1474, 1369, 1439, 1451, 1495, 1443, 1489, 1484, 1407, 1432, 1436, 1468, 1442, 1477, 1396, 1423, 1282, 1496, 1486, 1376, 1342, 1466, 1385, 1491, 1477, 1470, 1385, 1477, 1445, 1485, 1373, 1495, 1443, 1419, 1456, 1451, 1305, 1454, 1411, 1443, 1488, 1404, 1419, 1339, 1451, 1288, 1450, 1481, 1419, 1369, 1479, 1480, 1461, 1414, 1419, 1463, 1481, 1486, 1387, 1485, 1448, 1367, 1364, 1391, 1446, 1414, 1414, 1414, 1473, 1417, 1474, 1419, 1342, 1406, 1338, 1138

In [6]:
# chunk들의 최대값, 최소값

print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents[:-1]]))

1496
873


# 3. 쪼갠 문서를 임베딩 → 벡터 데이터베이스 저장

- 임베딩 모델 : upstage의 solar-embedding-1-large-query/passage
- 벡터데이터베이스(벡터 store) : chroma

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')

In [20]:
# embed_query : 한 문자열을 임베딩 벡터로 전환한 숫자list를 return

len(embedding.embed_query('소득세법은 다음과 같다')) # 길이가 길든 짧든 3072개의 숫자로 바꾸는 모델

3072

In [22]:
# embed_documents : 여러문자열을 임베딩 벡터로 전환

embedding_vector = embedding.embed_documents(
    [documents[0].page_content,
     documents[1].page_content]
)

In [26]:
print(len(embedding_vector), len(embedding_vector[0]), len(embedding_vector[1]))
print(embedding_vector[0][:10])

2 3072 3072
[0.014609415084123611, -0.00840421486645937, 0.0004996765055693686, 0.023311901837587357, 0.011843101121485233, 0.024633649736642838, -0.018176967278122902, 0.04655362293124199, -0.013123909942805767, 0.015170865692198277]


In [4]:
%%time

from langchain_chroma import Chroma

# 데이터 처음 저장할 때(두번 실행하면 계속 쌓이니까 한번만 실행하기)
database = Chroma.from_documents(
    documents=documents, # chunk
    embedding=embedding, # 임베딩 객체
    collection_name='tax-collection',    # oracle 같은곳의 table과 비슷. 생략시 이름은 랜덤
    persist_directory='./chroma_upstage' # 생략시 로컬 DB에 저장 안됨. 변수에 저장됨. 프로그램 종료시 DB날아감
)

CPU times: total: 5.61 s
Wall time: 38.9 s


In [28]:
# 이미 저장된 vector DB(store)를 사용할 때

database = Chroma(
    embedding_function=embedding,
    collection_name='tax-collection',
    persist_directory='./chroma'
)

In [9]:
result = database._collection.get(include=['embeddings', 'documents', 'metadatas'])
print('데이터 수 :', len(result['ids']))
print('문서 임베딩 차원 수 :', len(result['embeddings'][0]))
print('첫번째 임베딩 샘플 :', result['embeddings'][1])
print('첫번째 원본 :', result['documents'][1][:50])
print('첫번째 metadata :', result['metadatas'][1])

데이터 수 : 193
문서 임베딩 차원 수 : 3072
첫번째 임베딩 샘플 : [ 0.01674929 -0.01080192 -0.0115323  ...  0.00672168 -0.01151033
 -0.00994524]
첫번째 원본 : 1. 구성원 간 이익의 분배비율이 정하여져 있고 해당 구성원별로 이익의 분배비율이 확인되는
첫번째 metadata : {'source': 'data/소득세법(법률)(제21065호)(20260102).docx'}


# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrival)

In [5]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query=query,
                                           k=2) # 기본 k값은 4

In [6]:
# print('\n\n---\n\n'.join([doc.page_content for doc in retrived_docs]))
retrived_doc = '\n\n---\n\n'.join([doc.page_content for doc in retrived_docs])

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성(1)

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1-nano')

In [10]:
# prompt = f'[identity]-당신은 최구의 한국 소득세 전문가입니다. [context]를 참고해서 사용자의 질문에 답변해주세요. [context]내용은 다음과 같습니다. {retrived_doc} 질문 : {query}'
prompt = f'''너는 대한민국 세법(특히 소득세법)에 특화된 법령 분석 AI다.반드시 아래 원칙에 따라 질문에 답변한다:
1. 제공된 문서(Context)에 근거한 내용만 답변한다.
2. 문서에 명시되지 않은 내용은 추론하거나 일반화하지 않는다.
3. 문서에서 근거를 찾을 수 없는 경우, [제공된 소득세법 문서에는 해당 내용이 명시되어 있지 않습니다] 라고 답한다.
4. 실무적 조언이나 해석이 필요한 경우에도 조문을 우선 인용하고, 해석은 보조적으로만 제시한다. [context]는 다음과 같다. 
{retrived_doc} 
질문 : {query}'''

In [11]:
ai_message = llm.invoke(prompt)

In [12]:
print(ai_message.content)

[제공된 소득세법 문서에는 해당 내용이 명시되어 있지 않습니다]


In [48]:
ai_message.usage_metadata

{'input_tokens': 2131,
 'output_tokens': 509,
 'total_tokens': 2640,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성(2)

In [51]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
promptTemplate = ChatPromptTemplate([
    ('system', '당신은 최구의 한국 소득세 전문가입니다'),
    ('human',f'다음 문맥을 참고하여 질문에 답변하세요. 답을 모르면 모른다고 말하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{question}}/ 문맥 : {{context}}/ 답변 :')
])

In [52]:
promptTemplate

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 최구의 한국 소득세 전문가입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='다음 문맥을 참고하여 질문에 답변하세요. 답을 모르면 모른다고 말하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {question}/ 문맥 : {context}/ 답변 :'), additional_kwargs={})])

In [54]:
prompt = promptTemplate.invoke({
    'context':retrived_doc, # retrived_docs보다 추천
    'question':query
})

In [55]:
llm.invoke(prompt)

AIMessage(content='죄송하지만, 제공된 문맥에서 연봉 5천만원인 직장인의 소득세를 정확히 계산할 수 있는 정보가 포함되어 있지 않습니다. 소득세 계산에는 세율, 공제 및 세액 신고 요건 등이 필요하므로, 추가적인 정보가 필요합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 2151, 'total_tokens': 2216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CvasHGbq3gvn54QkZrKnXCDbwJ3sg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9b9f-7450-7de2-b741-e17052b0c679-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2151, 'output_tokens': 65, 'total_tokens': 2216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [56]:
# 위의 예제를 한번에

from  langchain_core.output_parsers import StrOutputParser

# llm.invoke(promptTemplate.invoke({'context':retrived_doc, 'question':query}))
output_parsers = StrOutputParser()
output_parsers.invoke(llm.invoke(promptTemplate.invoke({'context':retrived_doc, 'question':query})))

'연봉 5천만원인 직장인의 소득세는 정확하게 계산하기 위해서는 다양한 공제 항목과 세율을 고려해야 합니다. 기본적으로 근로소득에 대한 세율이 적용되며, 대략적인 소득세 계산기는 사용 가능하지만, 구체적인 액수가 필요하다면 세무전문가의 도움을 받는 것이 좋습니다. 현재 문맥에서 정확한 소득세 액수를 제시할 수는 없습니다.'

# 6. Langchain으로 답변 생성

In [58]:
# 위의 예제를 langchain으로 답변 생성

rag_chain = promptTemplate | llm | output_parsers
rag_chain.invoke({'context':retrived_doc, 'question':query})

'연봉 5천만원인 직장인의 소득세는 개별적인 세액공제 및 세율 적용에 따라 달라지므로 정확한 계산이 필요합니다. 대략적으로 계산하면, 근로소득세율에 따라 약 400만원에서 500만원 사이의 소득세가 발생할 수 있습니다. 하지만 정확한 세액은 개인의 상황에 따라 다를 수 있으니, 세무 전문가의 상담이 필요합니다.'

## langchain 전달

- query → retriver 전달 → prompt에 context 삽입
- smith.langchain.com에서 key 생성 후 .env에 LANGCHAIN_API_KEY 추가

In [13]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage
from dotenv import load_dotenv
load_dotenv()

# 1. LLM과 임베딩 초기화
# llm = ChatOpenAI(model='gpt-4.1-mini')
llm = ChatUpstage(model='solar-pro2')
embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')

# 2. vertor store load
vectorstore = Chroma(embedding_function=embedding,
                     collection_name='tax-collection',
                     persist_directory='./chroma_upstage')

# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type='similarity',
                                     search_kwargs={'k':4})

# 4. 프롬프트 템플릿
template = f'당신은 최고의 한국 소득세 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
prompt = ChatPromptTemplate.from_template(template)

# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])

In [14]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {'query':'~'} → '~'
rag_chain = ({'context':retriever | format_documents, 'query':RunnablePassthrough()} # 질문 그대로 전달
             | prompt # prompt에 context와 query 변수 주입
             | llm    # llm에 prompt invoke
             | StrOutputParser()) 

# 7. 실행
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 소득세는 근로소득공제, 종합소득세율, 세액공제를 종합적으로 계산해야 하므로 제공된 문맥만으로는 정확한 금액을 산출할 수 없습니다. 추가 정보(가족 구성, 자녀 수, 기타 공제 항목 등)가 필요합니다.  \n\n예시:  \n1. **근로소득공제 적용**: 연봉 5천만원 기준 근로소득공제 약 1,200만원 공제 후 과세표준 3,800만원.  \n2. **종합소득세율**: 3,800만원에 6%~24% 누진세율 적용.  \n3. **세액공제**: 자녀세액공제(제59조의2), 근로소득세액공제(제59조) 등 추가 공제 필요.  \n\n정확한 금액은 개별 상황에 따라 세무사나 국세청 계산기를 이용해야 합니다.'